In [1]:
import xml.etree.ElementTree as ET 
import csv
xmlfile = '../IR_datasets/biology.stackexchange.com/PostLinks.xml'
tree = ET.parse(xmlfile)

In [2]:
root = tree.getroot()

In [3]:
def createCSV(root,postlinksFile):
    with open(postlinksFile,mode='w',encoding='utf-8') as postlinks:
        postlinks = csv.writer(postlinks, delimiter=',')
        postlinks.writerow(['Id','PostId','RelatedPostId','LinkTypeId'])
        for row in root.findall('row'):
            Id = row.get('Id')
            postId = row.get('PostId')
            relatedpostId = row.get('RelatedPostId')
            linktypeId = row.get('LinkTypeId')
            postlinks.writerow([Id,postId,relatedpostId,linktypeId])


In [4]:
createCSV(root,"../csv_files/postlinks.csv")

In [5]:
import pandas as pd
postlinksDF = pd.read_csv('../csv_files/postlinks.csv')

In [6]:
type(postlinksDF)

pandas.core.frame.DataFrame

In [7]:
postlinksDF

,Id,PostId,RelatedPostId,LinkTypeId
0,3211,344,186,1
1,3587,202,391,1
2,3918,431,344,1
3,4553,510,450,1
4,6008,649,115,1
5,6009,649,399,1
6,6054,653,651,1
7,6706,675,393,1
8,6707,677,653,1
9,6708,677,651,1


In [8]:
postlinksDF.shape

(5292, 4)

In [12]:
#final list of dict to keep track of related post ids and answer ids for each question id
postLinks = []
#to keep track of questions that were already encountered
posts = [] 

for index, row in postlinksDF.iterrows():
    temp = {'postID': None,'relatedID': [],'answerID': [] } 
    temp['postID'] = row['PostId']
    if(row['PostId'] in posts and len(posts)!=0):
        for i in range(0,len(posts)):
            if(row['PostId'] == postLinks[i]['postID']):
                postLinks[i]['relatedID'].append(row["RelatedPostId"])
                postLinks[i]['answerID'].append(row["Id"])
    else:  
        temp['relatedID'].append(row["RelatedPostId"])
        temp['answerID'].append(row["Id"])   
        posts.append(temp['postID'])
        postLinks.append(temp)       


In [13]:
len(postLinks)

3579

In [62]:
import pickle

#load the file of recommendations made by our model for each question
recommended_que= pickle.load(open("../models/ques_recommendations.dat","rb"))
recommended_ans = pickle.load(open("../models/ans_recommendations.dat","rb"))
combined = pickle.load(open("../models/que_ans_recommendations.dat","rb"))

In [93]:
title_recommended_que= pickle.load(open("../models/title_ques_recommendations.dat","rb"))
title_combined = pickle.load(open("../models/title_que_ans_recommendations.dat","rb"))

In [101]:
tags_recommended_que= pickle.load(open("../models/w_tags_ques_recommendations.dat","rb"))
tags_combined = pickle.load(open("../models/w_tags_que_ans_recommendations.dat","rb"))

In [42]:
len(recommended_que)

21594

In [102]:
#Function to evaluate recommendations for a question
#Evaluation made against the questions present in PostLinks.xml
#Weighted average of the number of questions from relatedID list are present in the list of recommendations 
#and the number of questions from relatedID list that are present in the top half of the list of recommendations

def evaluate(recommended,n):
    print(len(recommended))
    score  = 0
    for i in range(0,len(postLinks)):
        present = 0 
        top = 0
        count = len(postLinks[i]['relatedID'])
        if postLinks[i]['postID'] in recommended.keys():
            idx = 0
            for rec in recommended[postLinks[i]['postID']][0:n]:
                #print(rec)
                idx+=1
                if rec[0] in postLinks[i]['relatedID']:
                    present+=1
                    if (idx >= int(len(postLinks[i]['relatedID'])/2)):
                        top+=1
            if(present):
                score+= (0.3* (present/count)+ 0.7*(top/present))
    return (score/len(postLinks))
    

In [90]:
evaluate(recommended_que,30)

21594


0.11621745233438475

In [91]:
evaluate(recommended_ans,30)

17279


0.1320006586037604

In [92]:
evaluate(combined,30)

21594


0.1873227757154834

In [95]:
evaluate(title_recommended_que,30)

21594


0.08698702084913319

In [97]:
evaluate(title_combined,30)

21594


0.1898444298088056

In [103]:
evaluate(tags_recommended_que,30)

21594


0.08698702084913319

In [104]:
evaluate(tags_combined,30)

21594


0.1898444298088056